In [1]:
%load_ext autoreload
%autoreload 2



In [2]:
import torchreid


D:\Drone_object_tracking\torchreid\metrics\rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


In [3]:
from torchreid.data.datasets.video.VisDroneChilds.VisDroneChild_uav0000013_00000_v import VisDroneChild_uav0000013_00000_v
torchreid.data.register_video_dataset('VisDroneChild_uav0000013_00000_v', VisDroneChild_uav0000013_00000_v)

In [4]:
from torchreid.data.datasets.video.VisDroneChilds.VisDroneChild_uav0000013_01073_v import VisDroneChild_uav0000013_01073_v
torchreid.data.register_video_dataset('VisDroneChild_uav0000013_01073_v', VisDroneChild_uav0000013_01073_v)


In [5]:

from torchreid.data.datasets.video.VisDroneChilds.VisDroneChild_uav0000086_00000_v import VisDroneChild_uav0000086_00000_v
torchreid.data.register_video_dataset('VisDroneChild_uav0000086_00000_v', VisDroneChild_uav0000086_00000_v)


In [6]:

from torchreid.data.datasets.video.VisDroneChilds.VisDroneChild_uav0000137_00458_v import VisDroneChild_uav0000137_00458_v
torchreid.data.register_video_dataset('VisDroneChild_uav0000137_00458_v', VisDroneChild_uav0000137_00458_v)


In [7]:

from torchreid.data.datasets.video.VisDroneChilds.VisDroneChild_uav0000182_00000_v import VisDroneChild_uav0000182_00000_v
torchreid.data.register_video_dataset('VisDroneChild_uav0000182_00000_v', VisDroneChild_uav0000182_00000_v)


In [8]:

from torchreid.data.datasets.video.VisDroneChilds.VisDroneChild_uav0000339_00001_v import VisDroneChild_uav0000339_00001_v
torchreid.data.register_video_dataset('VisDroneChild_uav0000339_00001_v', VisDroneChild_uav0000339_00001_v)

In [9]:
datamanager = torchreid.data.VideoDataManager(
    root='D:\\Drone_object_tracking\\reid-data\\VisDrone2019-MOT-val',
    sources=['VisDroneChild_uav0000137_00458_v', 'VisDroneChild_uav0000339_00001_v'],
    targets = ['VisDroneChild_uav0000182_00000_v'],
    height = 180,    #resizes images to this
    width=90,
    batch_size_train = 12,#how many tracklets in a batch. Small batch size may cause error because 1 id is nor enough to form tripleet loss. Batch_size >=  num_instances*3
    batch_size_test = 12,#how many tracklets in a batch
    seq_len = 10,    # how many frmaes in a tracklet
    num_instances = 3, #how many tracklets per id in a batch
    train_sampler='CustomDatasetSampler'
)

Building train transforms ...
+ resize to 180x90
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 180x90
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
self.video_name uav0000137_00458_v
=> Loaded VisDroneChild_uav0000137_00458_v
  -------------------------------------------
  subset   | # ids | # tracklets | # cameras
  -------------------------------------------
  train    |   176 |        1537 |         1
  query    |   131 |         256 |         1
  gallery  |   171 |        1281 |         1
  -------------------------------------------
self.video_name uav0000339_00001_v
=> Loaded VisDroneChild_uav0000339_00001_v
  -------------------------------------------
  subset   | # ids | # tracklets | # cameras
  -------------------------------------------
  train    |    80 |         835

In [10]:
model = torchreid.models.build_model(
    name='mobilenetv2_x1_4',
    num_classes=datamanager.max_train_pid+1,#КОСТЫЛЬ, возможно),256 - ERROR
    loss='triplet',
    pretrained=True
    
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0003
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer, lr_scheduler='multi_step', stepsize=[15, 25, 35]
)


D:\Drone_object_tracking\torchreid\models\mobilenetv2.py:270: UserWarning: The imagenet pretrained weights need to be manually downloaded from https://mega.nz/#!RGhgEIwS!xN2s2ZdyqI6vQ3EwgmRXLEW3khr9tpXg96G9SUJugGk
  warnings.warn(


In [11]:
engine = torchreid.engine.VideoTripletEngine(
    datamanager, model,optimizer,margin=0.4, weight_t=1, weight_x=0,scheduler=scheduler,
    pooling_method='avg'
)

In [12]:
"""engine = torchreid.engine.VideoSoftmaxEngine(
    datamanager, model,optimizer,scheduler=scheduler,
    pooling_method='avg'
)"""

"engine = torchreid.engine.VideoSoftmaxEngine(\n    datamanager, model,optimizer,scheduler=scheduler,\n    pooling_method='avg'\n)"

In [13]:


engine.run(
    max_epoch=150,
    save_dir='log/mobilenetv2_x1_4-visdrone_start_22_02_9h_44m',
    print_freq=30,  #if (self.batch_idx + 1) % print_freq == 0:, where for self.batch_idx, data in enumerate(self.train_loader):
    eval_freq=2,
    #start_epoch=start_epoch,
    #test_only=True,
    #visrank=True
    #start_epoch=start_epoch,
    
)
#loss_t - triplet loss
#loss_x - cross entropy loss
#validation on same as training - 100% results
#sampler selects tracklets to batches??
""" used like:
self.train_loader = torch.utils.data.DataLoader(
            trainset,
            sampler=build_train_sampler(
                trainset.train,
                train_sampler,
                batch_size=batch_size_train,
                num_instances=num_instances,
                num_cams=num_cams,
                num_datasets=num_datasets
            ),
            ...
        )
"""

"""
Если вообще не учить перед валидацией - 95+
Если хотябы одну эпоху на любом датасете будет хуже
В QUERY и GALLERY одну и тоже может попасть
"""
#HARD MINE YRIPLET LOSS distAP not considered


=> Start training
epoch: [1/150][30/184]	time 6.109 (6.884)	data 0.000 (0.712)	eta 2 days, 4:43:18	loss_t 0.5570 (0.6864)	lr 0.000300
epoch: [1/150][60/184]	time 6.031 (6.453)	data 0.000 (0.356)	eta 2 days, 1:21:50	loss_t 0.6828 (0.6696)	lr 0.000300
epoch: [1/150][90/184]	time 5.812 (6.307)	data 0.000 (0.237)	eta 2 days, 0:11:36	loss_t 0.3672 (0.6337)	lr 0.000300
epoch: [1/150][120/184]	time 6.031 (6.230)	data 0.000 (0.178)	eta 1 day, 23:33:17	loss_t 0.4736 (0.6101)	lr 0.000300
epoch: [1/150][150/184]	time 6.125 (6.191)	data 0.000 (0.142)	eta 1 day, 23:12:28	loss_t 0.2252 (0.5897)	lr 0.000300
epoch: [1/150][180/184]	time 6.062 (6.174)	data 0.000 (0.119)	eta 1 day, 23:01:35	loss_t 0.4542 (0.5695)	lr 0.000300
epoch: [2/150][30/184]	time 6.078 (7.282)	data 0.000 (1.447)	eta 2 days, 7:23:52	loss_t 0.5182 (0.4464)	lr 0.000300
epoch: [2/150][60/184]	time 6.031 (6.652)	data 0.000 (0.724)	eta 2 days, 2:32:40	loss_t 0.6690 (0.4509)	lr 0.000300
epoch: [2/150][90/184]	time 6.031 (6.418)	data 0.00

KeyboardInterrupt: 

In [ ]:
#THIS ALL WORKS ONLY FOR TRIPLET LOSS